# Proyecto 1 - MLOPS

**Integrantes:**

- Camilo Diaz Granados Nobman
- Luis Carlos Fernandez Vargas
- Daniel Alejandro Ruiz


Este proyecto busca evaluar la capacidad de crear un ambiente de desarrollo de machine learning
en el cual sea posible la ingesta, validación y transformación de datos, demostrando capacidad de
versionar código y ambiente de desarrollo.


Para el proyecto se utiliza una variante del conjunto de datos Tipo de Cubierta Forestal, de acuerdo con lo propuesto. Esto se puede utilizar **para entrenar un modelo que predice el tipo de cobertura forestal en función de variables cartográficas**.

## 1. Cargar el Dataset

Se procede a realizar la carga del dataset:

In [2]:
import os
import requests

# Cambiar la ruta del directorio a la nueva carpeta 'Data'
_data_root = '../data/covertype'
# Ruta al archivo de datos de entrenamiento
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Descargar datos
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    # URL del dataset
    url = 'https://docs.google.com/uc?export=download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)


## 2. Selección de Caracteristicas

Importamos las librerias necesarias para realizar la validación de los datos

In [3]:
import os
import requests
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
from sklearn.model_selection import train_test_split
print('TF version:', tf.__version__)
print('TFDV version:', tfdv.version.__version__)

2025-02-26 00:38:52.672351: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-26 00:38:52.678742: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-26 00:38:52.699019: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-26 00:38:52.737141: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-26 00:38:52.737205: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-26 00:38:52.764345: I tensorflow/core/platform/cpu_feature_guard.cc:

TF version: 2.16.2
TFDV version: 1.16.1


In [6]:
# Cargamos el dataset en un Dataframe desde la nueva ubicación
data_filepath = '../data/covertype/covertype_train.csv'
df = pd.read_csv(data_filepath, index_col=False)
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1


In [7]:
#Validamos la información de las caracteristicas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Elevation                           116203 non-null  int64 
 1   Aspect                              116203 non-null  int64 
 2   Slope                               116203 non-null  int64 
 3   Horizontal_Distance_To_Hydrology    116203 non-null  int64 
 4   Vertical_Distance_To_Hydrology      116203 non-null  int64 
 5   Horizontal_Distance_To_Roadways     116203 non-null  int64 
 6   Hillshade_9am                       116203 non-null  int64 
 7   Hillshade_Noon                      116203 non-null  int64 
 8   Hillshade_3pm                       116203 non-null  int64 
 9   Horizontal_Distance_To_Fire_Points  116203 non-null  int64 
 10  Wilderness_Area                     116203 non-null  object
 11  Soil_Type                           116

Teniendo en cuenta que dos (2) de las caracteristicas son categoricas, creamos subconjuntos de las caracteristicas con el fin de calificar cuales tienen un mayor impacto en la predicción de la etiqueta "Cover_Type"

In [9]:
#Creamos subconjuntos
numeric_features = df.select_dtypes(include=[int, float])
categorical_features = df.select_dtypes(include=[object])

print("Características numéricas:")
print(numeric_features.info())

print("\nCaracterísticas categóricas:")
print(categorical_features.info())

Características numéricas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 11 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           116203 non-null  int64
 1   Aspect                              116203 non-null  int64
 2   Slope                               116203 non-null  int64
 3   Horizontal_Distance_To_Hydrology    116203 non-null  int64
 4   Vertical_Distance_To_Hydrology      116203 non-null  int64
 5   Horizontal_Distance_To_Roadways     116203 non-null  int64
 6   Hillshade_9am                       116203 non-null  int64
 7   Hillshade_Noon                      116203 non-null  int64
 8   Hillshade_3pm                       116203 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  116203 non-null  int64
 10  Cover_Type                          116203 non-null  int64
dtypes: int64(11)
memory usage